# DeLorean
## Back to the future to estimate cars prices

### 1. Setup workspace

In [1]:
import pandas as pd
import numpy as np
import re

#Append folder for custom libraries
import sys
sys.path.append('libraries')

#Import custom library
import shinypanda

cars_price = pd.read_csv('data/cars_train.csv')

### 2. Explore the dataset

In [2]:
#Head of the dataframe
cars_price.head()

,Id,city,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,...,paint_color,lat,long,county_fips,county_name,state_fips,state_code,state_name,weather,price
0,559327,baltimore,2006.0,NaN,Scion tc,excellent,4 cylinders,gas,190000.0,clean,...,blue,39.287000,-76.647600,24510.0,Baltimore City,24.0,MD,Maryland,56.0,3200
1,1429566,carbondale,2018.0,dodge,charger sxt,NaN,NaN,gas,NaN,clean,...,NaN,37.720000,-89.215800,17077.0,Jackson,17.0,IL,Illinois,48.0,30620
2,931606,thumb,1997.0,ford,f 250 2 wheel dr pickup,fair,NaN,gas,NaN,clean,...,white,43.483300,-83.383500,26157.0,Tuscola,26.0,MI,Michigan,45.0,1800
3,1265412,laredo,2003.0,ram,NaN,NaN,8 cylinders,gas,NaN,clean,...,NaN,27.850069,-99.668883,48479.0,Webb,48.0,TX,Texas,67.0,4500
4,1133731,ocala,2000.0,dodge,1500,NaN,NaN,gas,NaN,clean,...,NaN,29.165000,-81.539900,12069.0,Lake,12.0,FL,Florida,65.0,1400


In [3]:
cars_price.count()

Id              469992
city            469992
year            468279
manufacturer    432732
make            450914
condition       278971
cylinders       281916
fuel            467148
odometer        316527
title_status    469284
transmission    467545
drive           289658
size            163217
type            278352
paint_color     280379
lat             469992
long            469992
county_fips     453983
county_name     453983
state_fips      453983
state_code      453983
state_name      469992
weather         453810
price           469992
dtype: int64

In [4]:
#Type of data
#cars_price.dtypes

In [5]:
#Descriptive stats
#cars_price.describe()

### 3. Data cleaning
#### In order to use sklearn algorithms, the entire dataset must contain operable numerical data

#### 3.1 Setup dataframe

In [6]:
#Remove all location info but State Code (could affect to taxes)
location_columns = ['city', 'lat', 'long', 'county_fips', 'county_name', 'state_fips', 'state_code', 'state_name']
cars_price = cars_price.drop(columns=location_columns, axis=1)

#Remove not relevant or unknown columns
cars_price = cars_price.drop(columns=['Id', 'weather'], axis=1)

#Remove unpopulated columns
cars_price = cars_price.drop(columns=['drive', 'size', 'type', 'paint_color', 'condition', 'cylinders', 'odometer'], axis=1)

#Rename columns
cars_price.rename(columns={'manufacturer':'brand', 'make':'model'}, inplace=True)

cars_price.head()

,year,brand,model,fuel,title_status,transmission,price
0,2006.0,NaN,Scion tc,gas,clean,automatic,3200
1,2018.0,dodge,charger sxt,gas,clean,automatic,30620
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,1800
3,2003.0,ram,NaN,gas,clean,automatic,4500
4,2000.0,dodge,1500,gas,clean,automatic,1400


#### 3.2 Clean 'brand' column

In [7]:
#Transform column type from 'object' to string
cars_price['brand'] = cars_price['brand'].astype('str') 

brands = sorted(cars_price.brand.unique())
print('Original brands list:', brands)

Original brands list: ['acura', 'alfa', 'alfa-romeo', 'aston', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chev', 'chevrolet', 'chevy', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'infinity', 'jaguar', 'jeep', 'kia', 'land rover', 'landrover', 'lexus', 'lincoln', 'mazda', 'mercedes', 'mercedes-benz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nan', 'nissan', 'pontiac', 'porche', 'ram', 'rover', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo', 'vw']


In [8]:
#Remove NaN
cars_price['brand'] = cars_price.brand.replace({r'nan': ''}, regex=True)

#Fix duplicity: 'alfa' and 'alfa-romeo'
cars_price['brand'] = cars_price.brand.replace({r'alfa-romeo': 'alfa'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'alfa': 'alfa-romeo'}, regex=True)

#Fix duplicity: 'aston' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'aston-martin': 'aston'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'aston': 'aston-martin'}, regex=True)

#Fix duplicity: 'chev', 'chevy' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'chevrolet': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chevy': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chev': 'chevrolet'}, regex=True)

#Fix duplicity: 'harley' and 'harley-davidson'
cars_price['brand'] = cars_price.brand.replace({r'harley-davidson': 'harley'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'harley': 'harley-davidson'}, regex=True)

#Fix misspelling: 'infinity' instead of 'infinity'
cars_price['brand'] = cars_price.brand.replace({r'infinity': 'infiniti'}, regex=True)

#Fix duplicity: 'rover', 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'landrover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land-rovex': 'land-rover'}, regex=True)

#Fix duplicity: 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'mercedes-benz': 'mercedes'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'mercedes': 'mercedes-benz'}, regex=True)

#Fix misspelling: 'porsche' instead of 'porche'
cars_price['brand'] = cars_price.brand.replace({r'porche': 'porsche'}, regex=True)

#Fix duplicity: 'vw' and 'volkswagen'
cars_price['brand'] = cars_price.brand.replace({r'vw': 'volkswagen'}, regex=True)

brands = sorted(cars_price.brand.unique())[1:] #[1:] is used to remove empty string ''
print('Clean brands list:', brands)

Clean brands list: ['acura', 'alfa-romeo', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'jaguar', 'jeep', 'kia', 'land-rover', 'lexus', 'lincoln', 'mazda', 'mercedes-benz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nissan', 'pontiac', 'porsche', 'ram', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo']


#### 3.3 Curate 'model' column

In [9]:
#Transform column type from 'object' to string and make it lowercase
cars_price['model'] = cars_price['model'].astype('str').str.lower()

#Fix detected misspelling
cars_price['model'] = cars_price.model.replace({r'cadillaj': 'cadillac'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'chervolet': 'chevrolet'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'cheverolet': 'chevrolet'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'volksvagen': 'volkswagen'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'volkswagon': 'volkswagen'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'ercedes': 'mercedes-benz'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'mercedes-benzbenz': 'mercedes-benz'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'mecedez': 'mercedes-benz'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'benz': 'mercedes-benz'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'nissaan': 'nissan'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'totota': 'toyota'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'suburu': 'subaru'}, regex=True)

cars_price['model'] = cars_price.model.replace({r'crystler': 'chrysler'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'crysler': 'chrysler'}, regex=True)

#Add new brands listed in 'model' to 'brands' list
brands.extend(['nissan', 'lamborghini', 'mustang', 'suzuki', 'kawasaki', 'scion', 'yamaha', 'thomas', 'oldsmobile'])
print('Expanded brands list:', brands)

Expanded brands list: ['acura', 'alfa-romeo', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'jaguar', 'jeep', 'kia', 'land-rover', 'lexus', 'lincoln', 'mazda', 'mercedes-benz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nissan', 'pontiac', 'porsche', 'ram', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo', 'nissan', 'lamborghini', 'mustang', 'suzuki', 'kawasaki', 'scion', 'yamaha', 'thomas', 'oldsmobile']


In [10]:
#Split 'model' data
cars_price['extracted_brand'] = cars_price["model"].str.split(" ", n=-1, expand=False) 

#When 'model' contains a string that already exists in 'brands', export it in a new column
def brand_in_model(row):    
    brand = list(set(row['extracted_brand']).intersection(brands))
    return brand

#Apply the function and convert column to string again
cars_price['extracted_brand'] = cars_price.apply(lambda row: brand_in_model(row), axis=1)
cars_price['extracted_brand'] = cars_price['extracted_brand'].apply(lambda brand: ','.join(map(str, brand)))

#Split 'extracted brand' to preserve only the first match with brands list
split_brand = cars_price['extracted_brand'].str.split(',', n=1, expand=True)
cars_price['extracted_brand'] = split_brand[0]

found_brands = sorted(cars_price['extracted_brand'].unique())[1:] #[1:] is used to remove empty string ''
print('Brands found in \'model\' column:', found_brands)

Brands found in 'model' column: ['cadillac', 'chevrolet', 'chrysler', 'kawasaki', 'lamborghini', 'mercedes-benz', 'mustang', 'nissan', 'oldsmobile', 'porsche', 'scion', 'subaru', 'suzuki', 'thomas', 'toyota', 'volkswagen', 'yamaha']


In [11]:
#Show new content in table
brand_in_model = cars_price['extracted_brand'] != ''
empty_brand = cars_price['brand'] == ''

cars_price[empty_brand & brand_in_model].head()

,year,brand,model,fuel,title_status,transmission,price,extracted_brand
0,2006.0,,scion tc,gas,clean,automatic,3200,scion
26,2007.0,,chevrolet,gas,clean,automatic,260000,chevrolet
39,2009.0,,porsche cayenne turbo,gas,clean,automatic,8500,porsche
43,2013.0,,scion fr-s,gas,lien,manual,16000,scion
156,2009.0,,volkswagen jetta,diesel,rebuilt,manual,5900,volkswagen


In [12]:
#Remove brands from 'model' column
for brand in brands:
    cars_price['model'] = cars_price.model.replace({brand: ''}, regex=True)

#Clean symbols and strip 'model' column
cars_price['model'] =  [re.sub(r'[^\s\w]','', str(model)) for model in cars_price['model']]
cars_price['model'] = cars_price['model'].str.strip()

#Extract years from 'model' column
pattern = '(19[0-9][0-9]|20[0-1][0-9])'
cars_price['extracted_year'] = cars_price.model.str.extract(pattern, expand=False)

#Remove years from 'model' column
cars_price['model'] = [re.sub(pattern,'', str(model)) for model in cars_price['model']]

#Show new content in table
brand_in_model = cars_price['extracted_brand'] != ''
empty_brand = cars_price['brand'] == ''
year_in_model = cars_price['extracted_year'].notnull()

cars_price[empty_brand & brand_in_model & year_in_model].head()

,year,brand,model,fuel,title_status,transmission,price,extracted_brand,extracted_year
382,2005.0,,xa,gas,clean,manual,2125,scion,2005
1144,2007.0,,jetta,gas,clean,automatic,5900,volkswagen,2007
2575,2015.0,,frs,gas,rebuilt,automatic,11000,scion,2015
3951,2011.0,,jetta se,gas,clean,automatic,6990,volkswagen,2011
4470,2004.0,,alero,gas,clean,automatic,2100,oldsmobile,2004


In [13]:
#Convert year-columns types to string
cars_price['year'] = cars_price['year'].astype(str)
cars_price['extracted_year'] = cars_price['extracted_year'].astype(str)

#Replace 'nan' year with empty strings
cars_price['year'] =  cars_price['year'].replace({r'nan': ''}, regex=True)

#Replace 'extracted_year'
cars_price['extracted_year'] =  cars_price['extracted_year'].replace({r'nan': ''}, regex=True)

#Show new content in table
empty_year = cars_price['year']==''
extracted_year = cars_price['extracted_year']!=''

cars_price[empty_year & extracted_year].head()

,year,brand,model,fuel,title_status,transmission,price,extracted_brand,extracted_year
13501,,,plymouth concourd,gas,clean,manual,6500,,1951
55154,,,tundra limited trd,gas,clean,automatic,21000,,2011
82086,,,corvette,diesel,missing,automatic,6000,,1988
136274,,,school bus,diesel,clean,automatic,4500,thomas,2001
137376,,,,gas,clean,automatic,39500,,1970


In [14]:
#Where 'brand' is empty, replace it with 'extracted_brand'
def complete_brand(row):
    
    brand_doesnt_exist = row['brand']==''
    extracted_brand_exists = row['extracted_brand']!=''
    
    if brand_doesnt_exist and extracted_brand_exists:
        return row['extracted_brand']
    else:
        return row['brand']

cars_price['brand'] = cars_price.apply(lambda row: complete_brand(row), axis=1)

#Where 'year' is empty, replace it with 'extracted_year'
def complete_year(row):
    
    year_doesnt_exist = row['year']==''
    extracted_year_exists = row['extracted_year']!=''
    
    if year_doesnt_exist and extracted_year_exists:
        return row['extracted_year']
    else:
        return row['year']

cars_price['year'] = cars_price.apply(lambda row: complete_year(row), axis=1)

#Remove 'extracted_brand' and 'extracted_year' columns
cars_price = cars_price.drop(['extracted_brand', 'extracted_year'], axis=1)

#Replace empty fields in 'brand' and 'year' columns with 'unknown'
cars_price['brand'] = cars_price['brand'].replace('', 'unknown')
cars_price['year'] = cars_price['year'].replace('', 'unknown')

cars_price.head()

,year,brand,model,fuel,title_status,transmission,price
0,2006.0,scion,tc,gas,clean,automatic,3200
1,2018.0,dodge,charger sxt,gas,clean,automatic,30620
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,1800
3,2003.0,ram,nan,gas,clean,automatic,4500
4,2000.0,dodge,1500,gas,clean,automatic,1400


#### 3.4 Clean 'fuel'

In [15]:
print ('Original fuel types:', list(cars_price['fuel'].unique()))

Original fuel types: ['gas', 'diesel', 'other', 'hybrid', nan, 'electric']


In [16]:
#Fill 'nan' with 'unknown'
cars_price['fuel'] = cars_price['fuel'].fillna(value='unknown')

print ('Current fuel types:', list(cars_price['fuel'].unique()))

Current fuel types: ['gas', 'diesel', 'other', 'hybrid', 'unknown', 'electric']


#### 3.5 Clean 'title_status'

In [17]:
print ('Original title status:', list(cars_price['title_status'].unique()))

Original title status: ['clean', 'lien', 'salvage', 'missing', 'rebuilt', 'parts only', nan]


In [18]:
#Change 'other' and fill 'nan' with 'unknown'
cars_price['title_status'] = cars_price['title_status'].fillna(value='unknown')
cars_price['title_status'] = cars_price['title_status'].replace('other', 'unknown')

print ('Current title status:', list(cars_price['title_status'].unique()))

Current title status: ['clean', 'lien', 'salvage', 'missing', 'rebuilt', 'parts only', 'unknown']


#### 3.6 Clean 'tranmission'

In [19]:
print ('Original transmission types:', list(cars_price['transmission'].unique()))

Original transmission types: ['automatic', 'manual', nan, 'other']


In [20]:
#Fill 'nan' with 'unknown'
cars_price['transmission'] = cars_price['transmission'].fillna(value='unknown')

print ('Current transmission types:', list(cars_price['transmission'].unique()))

Current transmission types: ['automatic', 'manual', 'unknown', 'other']


### 3. Export model

#### 3.1 Last columns settings

In [21]:
#Drop model column
cars_price = cars_price.drop(columns=['model'], axis=1)

#Turn 'year' to int and fill 'nan'
cars_price['year'] = pd.to_numeric(cars_price['year'], errors='coerce')
mean_year = round(cars_price['year'].mean())
cars_price['year'] = cars_price['year'].fillna(mean_year)

#Get dummies
clean_train_dataset = pd.get_dummies(cars_price)
clean_train_dataset.head()

,year,price,brand_acura,brand_alfa-romeo,brand_aston-martin,brand_audi,brand_bmw,brand_buick,brand_cadillac,brand_chevrolet,...,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,title_status_unknown,transmission_automatic,transmission_manual,transmission_other,transmission_unknown
0,2006.0,3200,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2018.0,30620,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1997.0,1800,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2003.0,4500,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2000.0,1400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


#### 3.2 Export file

In [25]:
clean_train_dataset.to_csv(r'data/clean_train_dataset.csv', index=False)